In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from PIL import Image
import torchvision.transforms as transforms
import torch
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

In [ ]:
def hot_encode(x):
    num_classes = 2
    return torch.eye(num_classes)[x]

In [ ]:
data_transforms = transforms.Compose([transforms.Resize((100,100)),             # resize the input to 224x224
                                      transforms.ToTensor(),              # put the input to tensor format
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # normalize the input
                                    ])
## load dataset from the folder
dataset = datasets.ImageFolder("dataset",transform=data_transforms,target_transform=hot_encode)

## split data
train_size = int(np.floor(len(dataset)*0.8))
test_size = len(dataset) - train_size
train_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size])

## Build Neural Network

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self,x):
        return x        
    
class classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4608,265)
        self.leakyrelu = nn.LeakyReLU(0.001)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(265,128)
        self.fc3 = nn.Linear(128,2)
        
    def forward(self,x):
        x = self.dropout(self.leakyrelu(self.fc1(x)))
        x = self.dropout(self.leakyrelu(self.fc2(x)))
        x = self.fc3(x)
        return F.softmax(x,dim = 1)

In [ ]:
device = "cuda" if torch.cuda.is_available()  else "cpu"

In [ ]:
vgg16 = models.vgg16(pretrained=True)
for param in vgg16.parameters():
    param.requires_grad = False
vgg16.avgpool = Identity()
vgg16.classifier = classifier()
vgg16.to(device)

## Training

In [ ]:
EPOCH = 10
LR = 0.001
BATCH_SIZE = 64
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(vgg16.parameters(), lr=LR)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
def train(model):
    for epoch in range(EPOCH):
        losses = []
        accuracies = []
        for batch in tqdm(train_loader):
            X_batch ,Y_batch = batch[0].view(-1,3,100,100).to(device), batch[1].to(device)

            output = model(X_batch)
            loss = loss_function(output, Y_batch)
            losses.append(loss)
            vgg16.zero_grad()
            loss.backward()
            optimizer.step()
            ## find accuarcy
            matches = [torch.argmax(a) == torch.argmax(b) for a,b in zip(output, Y_batch)]
            accuracy = matches.count(True)/len(matches)
            accuracies.append(accuracy)
        print(f"Loss at epoch {epoch} is {sum(losses)/len(losses):.5f}")
        print(f"Accuracy at epoch {epoch} is {sum(accuracies)/len(accuracies):.5f}")


In [ ]:
train(vgg16)

In [ ]:
def test(net):
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(test_loader):
            X_batch ,Y_batch = batch[0].view(-1,3,100,100).to(device), batch[1].to(device)
            real_class = torch.argmax(Y_batch).to(device)
            net_out = vgg16(X_batch.view(-1, 3, 100, 100).to(device))[0]

            predicted_class = torch.argmax(net_out)
            if predicted_class == real_class:
                correct += 1
            total += 1
    print("Accuracy:", round(correct/total,3))

In [ ]:
test(vgg16)